In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
    "unsloth/gemma-2-2b-bnb-4bit",             # New small Gemma model!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b-it-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)



In [ ]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Extract the most confident information in the sentence below as much as possible, and express the relationships in RDF Triples that complement the existing RDF triples. Do not use information from common sense.
### Example:
Input:Marriott International, with Bill Marriott as a key leader, is the tenant of AC Hotel Bella Sky in Copenhagen.
Response:<T>AC_Hotel_Bella_Sky_Copenhagen<R>location<S>Copenhagen<T>AC_Hotel_Bella_Sky_Copenhagen<R>tenant<S>Marriott_International<T>Marriott_International<R>keyPerson<S>Bill_Marriott
### Input:
{}
### Response:
{}"""

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt.format(
        """Turn Me On is a 35.1 minute long album produced by Wharton Tiers that was followed by the album entitled Take it Off.""", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 500)

In [ ]:
import pandas as pd
import networkx as nx
import re
graph_gen_dataset = pd.read_csv('data/webnlg_test.csv', header=0, delimiter='\t')


def select_k(spectrum, minimum_energy = 0.9):
    running_total = 0.0
    total = sum(spectrum)
    if total == 0.0:
        return len(spectrum)
    for i in range(len(spectrum)):
        running_total += spectrum[i]
        if running_total / total >= minimum_energy:
            return i + 1
    return len(spectrum)

In [ ]:
similarities = []
for idx, data in graph_gen_dataset.iterrows():
    print(idx)
    if idx >= 50:
        break
    
    # ground truth
    graph1 = nx.DiGraph()

    for _triple in data["Triples"].split("<T>"):
        print(_triple)
        try:
            if _triple == "":
                continue
            split = re.split("<R>|<S>", _triple)
            subject = split[0]
            predicate = split[1]
            _object = split[2]

            graph1.add_edge(subject, _object)

        except Exception as e:
            print(f"NON-STANDARD TRIPLE {_triple} ({e})")
            continue
            
    # OpenAI
    
    # print(prompt.format(data["NL"], "").removesuffix("<unused0>"))
    inputs = tokenizer(
    [
        prompt.format(
            """Turn Me On is a 35.1 minute long album produced by Wharton Tiers that was followed by the album entitled Take it Off.""", # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    from transformers import TextStreamer
    out = model.generate(**inputs, max_new_tokens = 500)
    response = tokenizer.batch_decode(out)
    
    answer_text = response[0].split("### Response:\n")[1]
    graph2 = nx.DiGraph()
    for _triple in answer_text.split("<T>"):
        print(_triple)
        try:
            if _triple == "":
                continue
            split = re.split("<R>|<S>", _triple)
            subject = split[0]
            predicate = split[1]
            _object = split[2]

            graph2.add_edge(subject, _object)

        except Exception as e:
            print(f"NON-STANDARD TRIPLE {_triple} ({e})")
            continue
            
    # Similarity
    laplacian1 = nx.spectrum.laplacian_spectrum(graph1)
    laplacian2 = nx.spectrum.laplacian_spectrum(graph2)

    k1 = select_k(laplacian1)
    k2 = select_k(laplacian2)
    k = min(k1, k2)

    similarity = sum((laplacian1[:k] - laplacian2[:k])**2)
    print(similarity)
    similarities.append(similarity)

In [ ]:
import math
similarities

In [ ]:
import numpy as np
np.average(similarities)